<a href="https://colab.research.google.com/github/vars16/AI-Code-Explainer/blob/main/AI%20Code%20Explainer.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!pip install -q transformers accelerate bitsandbytes gradio torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["HF_TOKEN"] = ""
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config={"load_in_4bit": True}
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [3]:
EXAMPLES = {
    "Python": [
        'print("Hello, World!")',
        'def add(a, b):\n    return a + b',
        'for i in range(5):\n    print(i)'
    ],
    "Java": [
        'public class Hello {\n  public static void main(String[] args) {\n    System.out.println("Hello World");\n  }\n}',
        'int sum(int a, int b) {\n  return a + b;\n}'
    ],
    "C": [
        '#include <stdio.h>\nint main() {\n  printf("Hello, World!");\n  return 0;\n}',
        'int add(int a, int b) {\n  return a + b;\n}'
    ],
    "C++": [
        '#include <iostream>\nusing namespace std;\nint main() {\n  cout << "Hello World";\n  return 0;\n}',
        'int add(int a, int b) {\n  return a + b;\n}'
    ]
}


def explain_code(code, language):
    if not code.strip():
        return "Please enter code!"

    prompt = (
        f"Provide a concise explanation of this {language} code using exactly 3 bullet points:\n"
        "- Purpose: [state what the code does]\n"
        "- Key Components: [list main elements]\n"
        "- Output: [describe expected result]\n\n"
        f"Here is the code:\n{code}\n\nExplanation:"
    )

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=200)
        explanation = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "Explanation:" in explanation:
            explanation = explanation.split("Explanation:")[1].strip()

        if not all(x in explanation for x in ["Purpose:", "Key Components:", "Output:"]):
            explanation = "- Purpose:\n- Key Components:\n- Output:\n" + explanation

        return explanation
    except Exception as e:
        return f"Error: {str(e)}"

custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Fira+Code:wght@500&family=Montserrat:wght@700&display=swap');
body {
    background: linear-gradient(120deg, #18122B 0%, #393053 100%) fixed !important;
    min-height: 100vh;
    color: #e0c2ff !important;
    font-family: 'Fira Code', 'Montserrat', monospace, sans-serif !important;
    overflow-x: hidden;
}
.animated-bg {
    position: fixed;
    top: 0; left: 0; width: 100vw; height: 100vh;
    z-index: 0;
    background: radial-gradient(circle at 20% 30%, #a855f7 0%, transparent 60%),
                radial-gradient(circle at 80% 70%, #9333ea 0%, transparent 60%),
                linear-gradient(120deg, #18122B 0%, #393053 100%);
    opacity: 0.25;
    animation: bgmove 12s ease-in-out infinite alternate;
}
@keyframes bgmove {
    0% { filter: blur(0px); }
    100% { filter: blur(8px); }
}

h1, h2, label, .gr-button {
    color: #fff !important;
    text-shadow: 0 0 24px #a855f7, 0 0 2px #fff;
    font-family: 'Montserrat', 'Fira Code', monospace, sans-serif !important;
}

#code-col .gr-code, #code-col .gr-box, #code-col textarea {
    background: rgba(40, 20, 60, 0.92) !important;
    color: #f8f8ff !important;
    border: 2.5px solid #a855f7 !important;
    border-radius: 20px !important;
    box-shadow: 0 0 32px 8px #a855f7cc, 0 0 2px 1px #222 inset !important;
    font-size: 17px;
    font-family: 'Fira Code', monospace !important;
}
#code-col label[for^='Your Code'] {
    font-size: 1.2em;
    color: #fff !important;
     font-weight: bold;
    text-shadow:
        0 0 70px #a855f7,
        0 0 70px #a855f7,
        0 0 24px #9333ea,
        0 0 16px #c084fc,
        0 0 8px #fff !important;
    margin-bottom: 10px;
    letter-spacing: 1.2px;
}
#code-col .gr-code textarea {
    background: rgba(40, 20, 60, 0.98) !important;
    color: #f8f8ff !important;
    border: 2.5px solid #a855f7 !important;
    border-radius: 20px !important;
    box-shadow: 0 0 32px 8px #a855f7cc !important;
    padding: 20px !important;
    font-family: 'Fira Code', monospace !important;
}

#explanation-col .gr-textbox, #explanation-col .gr-box, #explanation-col textarea {
    background: rgba(30, 10, 40, 0.92) !important;
    color: #e0c2ff !important;
    border: 2.5px solid #c084fc !important;
    border-radius: 20px !important;
    box-shadow: 0 0 36px 10px #c084fcbb, 0 0 2px 1px #222 inset !important;
    font-size: 17px;
    font-family: 'Fira Code', monospace !important;
}
#explanation-col label[for^='Explanation'] {
    font-size: 1.2em;
    color: #fff !important;
    font-weight: bold;
    text-shadow: 0 0 18px #c084fc, 0 0 2px #fff;
    margin-bottom: 10px;
    letter-spacing: 1.2px;
}
#explanation-col .gr-textbox textarea {
    background: rgba(30, 10, 40, 0.98) !important;
    color: #e0c2ff !important;
    border: 2.5px solid #c084fc !important;
    border-radius: 20px !important;
    box-shadow: 0 0 36px 10px #c084fcbb !important;
    padding: 20px !important;
    font-family: 'Fira Code', monospace !important;
}

.gr-button {
    background: linear-gradient(90deg, #7e22ce, #9333ea) !important;
    border: none !important;
    color: #fff !important;
    font-weight: bold;
    padding: 16px 36px;
    border-radius: 16px !important;
    box-shadow: 0 0 40px #a855f7, 0 0 2px 1px #222 inset !important;
    text-shadow: 0 0 16px #a855f7;
    letter-spacing: 1.3px;
    font-size: 1.18em;
    margin-top: 18px !important;
    margin-bottom: 10px !important;
    transition: transform 0.2s, box-shadow 0.2s, background 0.2s;
}
.gr-button:hover {
    transform: scale(1.07) rotate(-1deg);
    background: linear-gradient(90deg, #9333ea, #7e22ce) !important;
    color: #fff !important;
    box-shadow: 0 0 60px 16px #a855f7cc, 0 0 2px 1px #222 inset !important;
}
.gr-dropdown, select {
    background-color: rgba(24, 18, 43, 0.92) !important;
    color: #c084fc !important;
    border: 2.5px solid #a855f7 !important;
    border-radius: 14px !important;
    box-shadow: 0 0 18px 4px #a855f7cc !important;
    font-size: 1.08em;
    font-family: 'Fira Code', monospace !important;
}
.gr-markdown h1, .gr-markdown h2 {
    color: #fff !important;
    text-shadow: 0 0 32px #a855f7, 0 0 2px #fff;
    letter-spacing: 2.5px;
    margin-bottom: 0.2em;
    font-family: 'Montserrat', 'Fira Code', monospace, sans-serif !important;
}
.gr-markdown strong { color: #a855f7 !important; }
.gr-markdown pre, .gr-markdown code {
    background: rgba(24, 18, 43, 0.92) !important;
    color: #c084fc !important;
    border-radius: 14px !important;
    box-shadow: 0 0 18px 4px #a855f7cc !important;
    padding: 16px 20px !important;
    font-family: 'Fira Code', monospace !important;
}
::selection { background: #a855f7; color: #fff; }

.flex-row {
    display: flex;
    flex-wrap: wrap;
    gap: 40px;
    justify-content: center;
    align-items: flex-start;
    margin-top: 32px;
    margin-bottom: 32px;
    z-index: 1;
}
.flex-col {
    flex: 1 1 400px;
    min-width: 340px;
    max-width: 650px;
    display: flex;
    flex-direction: column;
    gap: 18px;
}
@media (max-width: 1100px) {
    .flex-row { flex-direction: column; gap: 24px; }
    .flex-col { max-width: 100%; }
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.HTML('<div class="animated-bg"></div>')
    gr.Markdown("""
    <h1 style='font-size:2.7em;letter-spacing:2.5px;'><span style='color:#fff;text-shadow:0 0 32px #a855f7;'>AI Code Explainer</span></h1>
    <h2 style='color:#a855f7;text-shadow:0 0 24px #a855f7;font-size:1.4em;'>Understand Python, Java, C, or C++ code in plain English</h2>
    """)

    with gr.Row(elem_id="top-controls", equal_height=True):
        language_dropdown = gr.Dropdown(choices=list(EXAMPLES.keys()), value="Python", label="Select Language")
        example_selector = gr.Dropdown(choices=EXAMPLES["Python"], label="Select Example")

    with gr.Row(elem_id="main-content-row"):
        with gr.Column(elem_id="code-col"):
            code_input = gr.Code(label="Your Code", language="python", lines=16)
            explain_btn = gr.Button("Explain Code")
        with gr.Column(elem_id="explanation-col"):
            explanation_output = gr.Textbox(label="Explanation", lines=16, interactive=False)

    def update_examples(language):
        return gr.update(choices=EXAMPLES[language], value=EXAMPLES[language][0]), EXAMPLES[language][0]

    language_dropdown.change(fn=update_examples, inputs=language_dropdown, outputs=[example_selector, code_input])
    example_selector.change(fn=lambda code: code, inputs=example_selector, outputs=code_input)
    explain_btn.click(fn=explain_code, inputs=[code_input, language_dropdown], outputs=explanation_output)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://750e94488135d84be6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
